In [1]:
import pickle
import os
from perses.app.relative_point_mutation_setup import PointMutationExecutorRBD
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from pkg_resources import resource_filename
from simtk import unit
from perses.rjmc.topology_proposal import PointMutationEngineRBD

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [2]:
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [6]:
def generate_tleap_system2(tleap_prefix,
                        temperature=298 * unit.kelvin,
                        nonbonded_method=app.PME, 
                        constraints=app.HBonds, 
                        remove_cm_motion=False, 
                        hydrogen_mass=4.0 * unit.amu,
                        pressure=1.0 * unit.atmosphere,
                        barostat_update_frequency=50):

    """
    Generate a tleap system by 1) running tleap and 2) loading the tleap output prmtop and inpcrd files into openmm
    Parameters
    ----------
    tleap_prefix : str
        Prefix for tleap input and output files
    temperature : unit.kelvin, default 298 * unit.kelvin
        Temperature
    nonbonded_method : simtk.openmm.app.Forcefield subclass object default app.PME
        Nonbonded method
    constraints : simtk.openmm.app.Forcefield subclass object, default app.HBonds
        Bonds that should have constraints
    remove_cm_motion : boolean, default False
        Indicates whether to remove center of mass motion
    hydrogen_mass : unit.amu, default 4.0 * unit.amu
        Hydrogen mass
    pressure : unit.atmosphere, default 1.0
        Pressure
    barostat_update_frequency : int, default 50
        The frequency at which Monte Carlo pressure changes should be attempted (in time steps)
    Returns
    -------
    prmtop.topology : simtk.openmm.app.Topology object
        Topology loaded from the prmtop file
    inpcrd.positions : np.array
        Positions loaded from the inpcrd file
    system : simtk.openmm.System object
        Tleap generated system as an OpenMM object
    """
    
    # Run tleap
    os.system(f"tleap -s -f {tleap_prefix}.in > {tleap_prefix}.out")

    # Check if tleap was successful
    if not os.path.exists(f"{tleap_prefix}.prmtop"):
        raise Exception(f"tleap parametrization did not complete successfully, check {tleap_prefix}.out for errors")

    # Load prmtop and inpcrd files
    prmtop = app.AmberPrmtopFile(f"{tleap_prefix}.prmtop")
    inpcrd = app.AmberInpcrdFile(f"{tleap_prefix}.inpcrd")

    # Generate system
    system = prmtop.createSystem(
        nonbondedMethod=nonbonded_method,
        constraints=constraints,
        temperature=temperature,
        removeCMMotion=remove_cm_motion,
        hydrogenMass=hydrogen_mass
    )

#     # Add barostat
#     _logger.info("Added barostat!")
#     barostat = openmm.MonteCarloBarostat(pressure, temperature, barostat_update_frequency)
#     system.addForce(barostat)

    return prmtop.topology, inpcrd.positions, system


In [15]:
from simtk.openmm import app
from simtk import unit

import copy
import logging
import itertools
import os
import openeye.oechem as oechem
import numpy as np
import networkx as nx
import openmoltools.forcefield_generators as forcefield_generators
from perses.storage import NetCDFStorageView
from perses.rjmc.geometry import NoTorsionError
from perses.utils.rbd import edit_pdb_for_tleap, edit_tleap_in_inputs, edit_tleap_in_ions, generate_tleap_system
from functools import partial
from pkg_resources import resource_filename

try:
    from subprocess import getoutput  # If python 3
except ImportError:
    from commands import getoutput  # If python 2


class PointMutationEngineRBD2(PointMutationEngineRBD):
    def _generate_new_tleap_system(self, tleap_prefix, debug_dir, old_topology, new_topology, current_positions, is_complex):
        """
        Generates new system by: 1) mutating in pymol to get the new positions, 2) rearranging positions to match
        the atom order in the new_topology and copying solvent atoms from the old positions, 3) parametrizing the
        new system using tleap.
        
        Parameters
        ----------
        tleap_prefix : str
            Prefix for tleap input and output files
        debug_dir : str
            if specified, debug output files will be saved here
        old_topology : simtk.openmm.app.Topology object
            The old topology
        new_topology : simtk.openmm.app.Topology object
            The new topology
        current_positions : np.array
            The current positions
        is_complex : boolean
            Indicates whether the current system is apo or complex
        Returns
        -------
        new_positions : np.array
            The new positions
        new_system : simtk.openmm.System object
            The new system object        
        """
        
        # Prepare PDB for mutation by removing solvent and renumbering the tleap coordinates
        _logger.info("Prepping for mutation")
        name = 'rbd_ace2' if is_complex else 'rbd'
        prepped_pdb = os.path.join(debug_dir, f"2_{name}_for_mutation.pdb")
        self._prep_for_mutation(tleap_prefix, prepped_pdb, is_complex)

        # Generate PDB of new topology/positions using pymol
        _logger.info("Mutating")
        mutant_position = self._allowed_mutations[0][0] # assume only allowed_mutations only has one mutation
        mutant_residue = self._allowed_mutations[0][1] # assume only allowed_mutations only has one mutation
        mutant_pdb = os.path.join(debug_dir, f"3_{name}_mutant.pdb")
        if os.path.exists(mutant_pdb):
            os.system(f"rm {mutant_pdb}") # Otherwise, pymol will load this file in
        mutate_script = resource_filename('perses', 'data/rbd-ace2/3_mutate.py')
        #self._mutate(f"2_{name}_for_mutation.pdb", f'{self._chain_id}/{mutant_position}/', mutant_residue, name)
        os.system(f"python {mutate_script} {prepped_pdb} {mutant_pdb} {self._chain_id}/{mutant_position}/ {mutant_residue}")       
        # Prep PDBs for tleap
        _logger.info("Prepping PDBs for tleap")
        new_positions = self._prep_for_tleap(debug_dir, old_topology, new_topology, current_positions, int(mutant_position), mutant_residue, is_complex)
        
        # Edit tleap in file
        tleap_prefix = os.path.join(debug_dir, f"5_{name}_mutant_tleap")
        mutant_template = f'/data/chodera/zhangi/perses_benchmark/neq/14/91/debug/5_{name}_mutant_template_tleap.in'
        edit_tleap_in_inputs(mutant_template, tleap_prefix, debug_dir)

        # Generate system using tleap 
        _logger.info("Generating new system")
        _, _, new_system = generate_tleap_system2(os.path.join(debug_dir, f"5_{name}_mutant_tleap"))
       
        return new_positions, new_system
    
    def _prep_for_tleap(self, debug_dir, old_topology, new_topology, current_positions, mutant_position, mutant_residue, is_complex):
        """
        Given a mutated PDB, prepare a PDB for tleap input: 1) Rearrange the mutated PDB positions such that they 
        match the atom ordering in new_topology, 2) Copy the solvent positions from current_positions, 3) Save apo 
        RBD, apo ACE2 (for complex), solvent as separate PDBs.
        
        Parameters
        ----------
        debug_dir : str
            If specified, debug output files will be saved here
        old_topology : simtk.openmm.app.Topology object
            The old topology
        new_topology : simtk.openmm.app.Topology object
            The new topology
        current_positions : np.array
            The current positions
        mutant_position : int
            Position to mutate
        mutant_residue : str
            Three-letter code for the residue to mutate to. Example: For lysine, use 'LYS'
        is_complex : boolean
            Indicates whether the current system is apo or complex
        Returns
        -------
        new_positions : np.array
            The new positions
        """
        
        name = 'rbd_ace2' if is_complex else 'rbd'
        
        # Correct atom names in mutated PDB
        # Read lines
        mutant_pdb = os.path.join(debug_dir, f"3_{name}_mutant.pdb")
        with open(mutant_pdb, "r") as f:
            lines = f.readlines()

        # Iterate through lines, copying them over to new list of lines
        new_lines = []
        for line in lines:
            if 'TER' not in line and 'END' not in line and 'REMARK' not in line and 'TITLE' not in line and 'CRYST1' not in line and 'CONECT' not in line:
                current_res_name = line[17:20]
                current_res_id = int(line[23:26])
                if current_res_name == mutant_residue and current_res_id == mutant_position: # Fix atom names in mutant residue
                    atom = line[12:16]
                    if atom[0].isdigit():
                        atom_chars = len(atom[1:].strip(" "))
                        if atom_chars == 2: # if atom name is 2 characters
                            line = line[:12] + ' ' + line[13:15] + line[12] + line[16:]
                        elif atom_chars == 3:
                            line = line[:12] + line[13:16] + line[12] + " " + line[17:]  
                new_lines.append(line)

        # Update mutated PDB with corrected atom lines
        with open(mutant_pdb, 'w') as f:
            f.writelines(new_lines)

        # Load mutated (protonated) PDB
        mutated_pdb = app.PDBFile(mutant_pdb)
        mutated_n_atoms = mutated_pdb.topology.getNumAtoms()
        
        # Map atom indices from pymol PDB to atom indices in new_topology
        d_omm = {} # key: (atom name, residue id, chain id), value: atom index
        for atom_omm in new_topology.atoms():
            d_omm[(atom_omm.name, atom_omm.residue.id, atom_omm.residue.chain.id)] = atom_omm.index

        d_map = {} # key: atom index from pymol mutated PDB, value: atom index in new_topology
        for atom_pymol in mutated_pdb.topology.atoms():
            match_index = d_omm[(atom_pymol.name, atom_pymol.residue.id, atom_pymol.residue.chain.id)]
            d_map[atom_pymol.index] = match_index

        # Rearrange positions based on new_topology and add units to positions
        dim_1, dim_2 = np.array(mutated_pdb.positions).shape
        mutated_positions = unit.Quantity(np.zeros(shape=(dim_1, dim_2)), unit=unit.nanometers)
        positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in mutated_pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
        for k, v in d_map.items():
            mutated_positions[v] = positions[k]
        
#         # Copy solvent positions from old positions
#         solvent_atoms = [atom for atom in old_topology.atoms() if atom.residue.chain.id == 'Y']
#         first_solvent_atom = solvent_atoms[0].index
#         new_positions = unit.Quantity(np.zeros([mutated_n_atoms + len(solvent_atoms), 3]), unit=unit.nanometers)
#         new_positions[:mutated_n_atoms, :] = mutated_positions
#         new_positions[mutated_n_atoms:, :] = current_positions[first_solvent_atom:]
        new_positions = unit.Quantity(np.zeros([mutated_n_atoms, 3]), unit=unit.nanometers)
        new_positions[:mutated_n_atoms, :] = mutated_positions
 
        def save_apo(output_pdb, topology, positions, chains_to_keep):
            modeller = app.Modeller(topology, positions)
            to_delete = []
            for chain in modeller.topology.chains():
                if chain.id not in chains_to_keep:
                    to_delete.append(chain)
            modeller.delete(to_delete)
            app.PDBFile.writeFile(modeller.topology, modeller.positions, open(output_pdb, "w"), keepIds=True)

        # Save apo solute PDBs and then correct for tleap
        rbd_pdb = os.path.join(debug_dir, f"3_{name}_mutant_rbd_tleap.pdb")
        rbd_pdb_final = os.path.join(debug_dir, f'4_{name}_mutant_rbd_tleap_final.pdb')
        save_apo(rbd_pdb, new_topology, new_positions, ['R', 'X'])
        edit_pdb_for_tleap(rbd_pdb, rbd_pdb_final)
        
#         solvent_pdb = os.path.join(debug_dir, f"3_{name}_mutant_solvent_tleap.pdb")
#         solvent_pdb_final = os.path.join(debug_dir, f'4_{name}_mutant_solvent_tleap_final.pdb')
#         save_apo(solvent_pdb, new_topology, new_positions, ['Y'])
#         edit_pdb_for_tleap(solvent_pdb, solvent_pdb_final)
        
        if is_complex:
            ace2_pdb = os.path.join(debug_dir, f"3_{name}_mutant_ace2_tleap.pdb")
            ace2_pdb_final = os.path.join(debug_dir, f'4_{name}_mutant_ace2_tleap_final.pdb')
            save_apo(ace2_pdb, new_topology, new_positions, ['C', 'D', 'E'])
            edit_pdb_for_tleap(ace2_pdb, ace2_pdb_final, is_ace2=True)

        return new_positions

In [10]:
from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol, oechem
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine, PointMutationEngineRBD
from perses.rjmc.geometry import FFAllAngleGeometryEngine
from perses.utils.rbd import edit_pdb_for_tleap, edit_tleap_in_inputs, edit_tleap_in_ions, generate_tleap_system

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator
import os
from pkg_resources import resource_filename
import shutil
import tempfile

ENERGY_THRESHOLD = 1e-2
temperature = 298 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']
CL_CHARGE = unit.Quantity(value=-1.0, unit=unit.elementary_charge)
CL_SIGMA = unit.Quantity(value=0.4477656957373345, unit=unit.nanometer)
CL_EPSILON = unit.Quantity(value=0.14891274399999999, unit=unit.kilojoule_per_mole)
NA_CHARGE = unit.Quantity(value=1.0, unit=unit.elementary_charge)
NA_SIGMA = unit.Quantity(value=0.2439280690268249, unit=unit.nanometer)
NA_EPSILON = unit.Quantity(value=0.3658460312, unit=unit.kilojoule_per_mole)
O_CHARGE = unit.Quantity(value=-0.834, unit=unit.elementary_charge)
H_CHARGE = unit.Quantity(value=0.417, unit=unit.elementary_charge)


class PointMutationExecutorRBD2(PointMutationExecutorRBD):
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 phase='complex',
                 clean=False,
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 vanilla=True,
                 repartitioned=True,
                 debug_dir=None,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            clean : bool, default False
                whether to clean the PDB for tleap
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_file : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            vanilla : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            repartitioned : bool, default True
                whether to generate a RepartitionedHybridTopologyFactory
            debug_dir : str, default None
                if specified, debug output files will be saved here
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        # Make debug directory
        is_temp = False
        if debug_dir:
            if not os.path.exists(debug_dir):
                os.system(f"mkdir {debug_dir}")
        else:
            debug_dir = tempfile.mkdtemp()
            is_temp = True        

        ## Generate the old topology, positions, and system
        # Prep PDBs for tleap
        _logger.info("Editing PDBs for tleap")
        protein_name = os.path.basename(protein_filename)
        ligand_name = os.path.basename(ligand_input)
        protein_tleap = os.path.join(debug_dir, f"{protein_name[:-4]}_tleap.pdb")
        ligand_tleap = os.path.join(debug_dir, f"{ligand_name[:-4]}_tleap.pdb")
        if clean:
            edit_pdb_for_tleap(protein_filename, protein_tleap)
            edit_pdb_for_tleap(ligand_input, ligand_tleap)
        else:
            os.system(f"cp {protein_filename} {protein_tleap}")
            os.system(f"cp {ligand_input} {ligand_tleap}")
        
        # Edit tleap files
        _logger.info("Editing tleap.in input files")
        apo_tleap_prefix = os.path.join(debug_dir, "1_rbd_tleap")
        complex_tleap_prefix = os.path.join(debug_dir, "1_rbd_ace2_tleap")
        apo_template = "/data/chodera/zhangi/perses_benchmark/neq/14/91/debug/1_rbd_template_tleap.in"
        complex_template = "/data/chodera/zhangi/perses_benchmark/neq/14/91/debug/1_rbd_ace2_template_tleap.in"
        edit_tleap_in_inputs(apo_template, apo_tleap_prefix, debug_dir)
        edit_tleap_in_inputs(complex_template, complex_tleap_prefix, debug_dir)
        
#         _logger.info("Editing tleap.in number of ions")
#         edit_tleap_in_ions(apo_tleap_prefix)
#         edit_tleap_in_ions(complex_tleap_prefix)
        
        # Generating old systems
        _logger.info("Generating solvated old systems")
        apo_topology, apo_positions, apo_system = generate_tleap_system2(apo_tleap_prefix, nonbonded_method=app.NoCutoff)
        complex_topology, complex_positions, complex_system = generate_tleap_system2(complex_tleap_prefix, nonbonded_method=app.NoCutoff)
        
        # Correct the topologies
        _logger.info("Correcting tleap topologies")
        apo_topology_corrected = self._correct_topology(apo_topology)
        complex_topology_corrected = self._correct_topology(complex_topology, is_apo=False)
        
        # Format inputs for pipeline
        inputs = [[apo_topology_corrected, apo_positions, apo_system, apo_tleap_prefix, False], [complex_topology_corrected, complex_positions, complex_system, complex_tleap_prefix, True]]
    
        # Make system generator -- note this is only for system_generator.forcefield call in PointMutationEngine init
        molecules = []
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)
        
        # Run pipeline...
        htfs = []
        for (top, pos, sys, tleap_prefix, is_complex) in inputs:
            name = 'rbd_ace2' if is_complex else 'rbd'
            _logger.info(f"Generating topology proposal for {name}")
            point_mutation_engine = PointMutationEngineRBD2(wildtype_topology=top,
                                                         system_generator=self.system_generator,
                                                         chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                         max_point_mutants=1,
                                                         residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                         allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                         aggregate=True) # Always allow aggregation

            topology_proposal, new_positions = point_mutation_engine.propose(sys, top, pos, tleap_prefix, is_complex, debug_dir)

            # Fix naked charges in old and new systems
            for system in [topology_proposal.old_system, topology_proposal.new_system]:
                force_dict = {i.__class__.__name__: i for i in system.getForces()}
                if 'NonbondedForce' in [i for i in force_dict.keys()]:
                    nb_force = force_dict['NonbondedForce']
                    for i in range(nb_force.getNumParticles()):
                        charge, sigma, epsilon = nb_force.getParticleParameters(i)
                        if sigma == 0*unit.nanometer:
                            sigma = 0.6*unit.nanometer
                            nb_force.setParticleParameters(i, charge, sigma, epsilon)
                        if epsilon == 0*unit.kilojoule_per_mole:
                            epsilon = 0.0001*unit.kilojoule_per_mole
                            nb_force.setParticleParameters(i, charge, sigma, epsilon)
            
            # Check for charge change...
            charge_diff = point_mutation_engine._get_charge_difference(current_resname = topology_proposal._old_topology.residue_topology.name,
                                                                       new_resname = topology_proposal._new_topology.residue_topology.name)
            _logger.info(f"charge diff: {charge_diff}")
            if charge_diff != 0:
                new_water_indices_to_ionize = point_mutation_engine.get_water_indices(charge_diff, new_positions, topology_proposal._new_topology, radius=0.8)
                _logger.info(f"new water indices to ionize {new_water_indices_to_ionize}")
                PointMutationExecutorRBD._modify_new_system(new_water_indices_to_ionize, topology_proposal._new_system, charge_diff)
                PointMutationExecutorRBD._modify_atom_classes(new_water_indices_to_ionize, topology_proposal)

            factories = []
            if vanilla:
                repartitioned_endstate = None
                self.generate_htf(HybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex)
            if repartitioned:
                for repartitioned_endstate in [0, 1]:
                    self.generate_htf(RepartitionedHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex)


In [8]:
# Read args
# outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/77/"
# outdir = "/data/chodera/zhangi/perses_benchmark/repex/31/28/0"
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/91/"
residue = '501'
mutant = "TYR"

rbd_file = os.path.join(outdir, "0_rbd.pdb")
ace2_file = os.path.join(outdir, "0_ace2.pdb")


In [16]:
solvent_delivery = PointMutationExecutorRBD2(rbd_file,
                        'R',
                        residue,
                        mutant,
                        ligand_input=ace2_file,
                        ionic_strength=0.15*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        debug_dir=os.path.join(outdir, "debug/")
                       )



INFO:root:Editing PDBs for tleap
INFO:root:Editing tleap.in input files
INFO:root:Generating solvated old systems
INFO:root:Correcting tleap topologies
INFO:root:Generating topology proposal for rbd
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Adding new atoms
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{17: 8}, {20: 9}, {19: 9}, {18: 9}, {17: 9}, {16: 9}, {15: 9}, {14: 9}, {20: 8}, {19: 8}, {18: 8}, {14: 7}, {16: 8}, {15: 8}, {14: 8}, {20: 7}, {19: 7}, {18: 7}, {17: 7}, {16: 7}, {15: 7}]
INFO:proposal_generator:{17: 8}
INFO:proposal_generator:{20: 9}
INFO:proposal_generator:{19: 9}
INFO:proposal_generator:{18: 9}
INFO:proposal_generator:{17: 9}
INFO:proposal_generator:{16: 9}
INFO:proposal_generator:{15: 9}
INFO:proposal_generator:{14: 9}
INFO:proposal_generator:{20: 8}
INFO:proposal_generator:{19: 8}
INFO:proposal_generator:{18:

INFO:relative:Handling harmonic bonds...
INFO:relative:	handle_harmonic_bonds: looping through old_system to add relevant terms...
INFO:relative:	handle_harmonic_bonds: looping through new_system to add relevant terms...
INFO:relative:Handling harmonic angles...
INFO:relative:	handle_harmonic_angles: looping through old_system to add relevant terms...
INFO:relative:	handle_harmonic_angles: looping through new_system to add relevant terms...
INFO:relative:Handling torsion forces...
INFO:relative:	handle_periodic_torsion_forces: looping through old_system to add relevant terms...
INFO:relative:	handle_periodic_torsion_forces: looping through new_system to add relevant terms...
INFO:relative:Handling nonbonded forces...
INFO:relative:	handle_nonbonded: looping through all particles in hybrid...
INFO:relative:	handle_nonbonded: Handling Interaction Groups...
INFO:relative:	handle_nonbonded: Handling Hybrid Exceptions...
INFO:relative:	handle_nonbonded: Handling Original Exceptions...
INFO:

INFO:relative:Adding bond force terms...
INFO:relative:Adding angle force terms...
INFO:relative:Adding torsion force terms...
INFO:relative:Adding nonbonded force terms...
INFO:relative:	_add_nonbonded_force_terms: <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b0221882b40> > added to hybrid system
INFO:relative:	_add_nonbonded_force_terms: nonbonded_method is NoCutoff
INFO:relative:	_add_nonbonded_force_terms: 0 added to standard nonbonded force
INFO:relative:	_add_nonbonded_force_terms: 0 added to sterics_custom_nonbonded force
INFO:relative:	_add_nonbonded_force_terms: <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b0221882ba0> > added to hybrid system
INFO:relative:Handling harmonic bonds...
INFO:relative:	handle_harmonic_bonds: looping through old_system to add relevant terms...
INFO:relative:	handle_harmonic_bonds: looping through new_system to add relevant terms

In [17]:
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)